# How to use the approximate model

You have already seen a first example of the [setup_sum_of_squares_problem](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.costs.setup_sum_of_squares_problem.html) function in the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial.
In this guide, we are going to revisit that function in more detail.

## Setting up a simple model problem

For this guide, we will reuse the simple model problem from the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial; the Ising model on a line of 10 sites:

$$
\hat{\mathcal{H}}_{\text{Ising}} = \sum_{i=1}^{9} J_{i,(i+1)} Z_i Z_{(i+1)} + \sum_{i=1}^{10} h_i X_i \, ,
$$

where $J$ is the coupling strength between two sites and $h$ is the external magnetic field.

In [1]:
from qiskit.transpiler import CouplingMap
from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian

coupling_map = CouplingMap.from_line(10, bidirectional=False)

hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(0.0, 0.0, 1.0),
    ext_magnetic_field=(0.4, 0.0, 0.0),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIZZI', 'IIIIIZZIII', 'IIIZZIIIII', 'IZZIIIIIII', 'IIIIIIIIZZ', 'IIIIIIZZII', 'IIIIZZIIII', 'IIZZIIIIII', 'ZZIIIIIIII', 'IIIIIIIIIX', 'IIIIIIIIXI', 'IIIIIIIXII', 'IIIIIIXIII', 'IIIIIXIIII', 'IIIIXIIIII', 'IIIXIIIIII', 'IIXIIIIIII', 'IXIIIIIIII', 'XIIIIIIIII'],
              coeffs=[1. +0.j, 1. +0.j, 1. +0.j, 1. +0.j, 1. +0.j, 1. +0.j, 1. +0.j, 1. +0.j,
 1. +0.j, 0.4+0.j, 0.4+0.j, 0.4+0.j, 0.4+0.j, 0.4+0.j, 0.4+0.j, 0.4+0.j,
 0.4+0.j, 0.4+0.j, 0.4+0.j])


## Choosing $k_j$

In this guide, we are going to focus on the tweaks that we can apply to the [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) and its solver that are returned by the [setup_sum_of_squares_problem](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.costs.setup_sum_of_squares_problem.html). Therefore, we are not concerned with choosing any specific $k_j$ values and can simply reuse the ones from the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial.

In [2]:
trotter_steps = (8, 12, 19)

## Computing $x$ analytically

First, we have to construct our LSE, $Ax=b$. Once again, we reuse the settings from the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial.

In [3]:
from qiskit_addon_mpf.static import setup_static_lse

lse = setup_static_lse(trotter_steps, order=2, symmetric=True)

Next, we compute the analytical solution of $x$ and its L1-norm as our reference values.

In [4]:
coeff_analytical = lse.solve()
print(coeff_analytical)

[ 0.17239057 -1.19447005  2.02207947]


In [5]:
import numpy as np

print(np.linalg.norm(coeff_analytical, ord=1))

3.388940092165899


## Setting up the approximate model

Now, we will construct the approximate model.
The idea of using approximate solutions $\tilde{x}$ in order to ensure a smaller L1-norm was proposed by [Zhuk et al., 2023].
The model is already explained in detail in the API documentation of [setup_sum_of_squares_problem](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.costs.setup_sum_of_squares_problem.html) so we refrain from repeating it here.
Suffice to say, that the model consists of two constraints which ensure that the coefficients, $\tilde{x}$, sum to $1$ and it enforces their L1-norm to be smaller than the user-provided `max_l1_norm` value.
The optimization objective is to minimize the deviation of $A\tilde{x}=b$.

We can see all of this in the output below.
Here, we are setting `max_l1_norm=3` because we already know that the L1-norm of the exact solution is approximately $3.4$ and we want to find a $\tilde{x}$ that has a smaller L1-norm.

[Zhuk et al., 2023]: https://arxiv.org/abs/2306.12569

In [6]:
from qiskit_addon_mpf.costs import setup_sum_of_squares_problem

model, coeff_var = setup_sum_of_squares_problem(lse, max_l1_norm=3)
print(model)

minimize quad_over_lin(Vstack([1. 1. 1.] @ x + -1.0, [0.015625   0.00694444 0.00277008] @ x + -0.0, [2.44140625e-04 4.82253086e-05 7.67336039e-06] @ x + -0.0), 1.0)
subject to Sum(x, None, False) == 1.0
           norm1(x) <= 3.0


## Solving the model with default settings

First, let us see what happens when we solve the `model` with only default settings.

In [7]:
model.solve(verbose=True)

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Sep 05 09:28:06 AM: Your problem has 3 variables, 2 constraints, and 0 parameters.
(CVXPY) Sep 05 09:28:06 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 05 09:28:06 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 05 09:28:06 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 05 09:28:06 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 05 09:28:06 AM: Compiling problem (target solver=OSQP).
(CVXP

4.092215822096244e-09

We can see from the final value of the cost function that the sum of squares of $A\tilde{x}-b$ is almost zero.

In the output above, we also get a lot more information from [cvxpy.Problem.solve](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem.solve), some of which we will get back to later.

For now, let us inspect the final values of $\tilde{x}$ and its L1-norm:

In [8]:
coeff_approx_plain = coeff_var.value
print(coeff_approx_plain)
print(np.linalg.norm(coeff_approx_plain, ord=1))

[-0.39646076  0.55556835  0.84089365]
1.7929227539822414


We can clearly see that the L1-norm has decreased compared to the analytical solution that we have computed before.

Nonetheless, let us see if we can tweak our solution a bit.
In the end, we will compare the results obtained for this set of coefficients with all the other ones.

## Tweaking the approximate model

Rather than tweaking the input parameters like the chosen `trotter_steps` and `max_l1_norm` values, we will focus on the solver settings in this guide.

### Using a different solver

Notice, that we used the default solver method: `OSQP`. We can change that solver to see whether this has any effect.

In [9]:
model.solve(verbose=True, solver="CLARABEL")

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Sep 05 09:28:07 AM: Your problem has 3 variables, 2 constraints, and 0 parameters.
(CVXPY) Sep 05 09:28:07 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 05 09:28:07 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 05 09:28:07 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 05 09:28:07 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 05 09:28:07 AM: Compiling problem (target solver=CLARABEL).
(

1.8479854472132682e-09

In [10]:
coeff_approx_clarabel = coeff_var.value
print(coeff_approx_clarabel)
print(np.linalg.norm(coeff_approx_clarabel, ord=1))

[-0.21284352 -0.00792683  1.22077035]
1.4415406931636157


Indeed, the outcome has changed. But now we have one major contribution from the largest $k_j$ value which is also not ideal.

An exhaustive search of the various available solvers is beyond the scope of this guide. Instead we encourage you to try this yourself.

### Changing the convergence thresholds

Rather than changing the solver altogether, we can also tweak the convergence parameters.
For the `OSQP` solver the relevant ones are called `eps_abs` and `eps_rel` and they are both set to `1e-5` by default.

In [11]:
model.solve(verbose=True, eps_abs=1e-8, eps_rel=1e-8)

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Sep 05 09:28:07 AM: Your problem has 3 variables, 2 constraints, and 0 parameters.
(CVXPY) Sep 05 09:28:07 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 05 09:28:07 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 05 09:28:07 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 05 09:28:07 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 05 09:28:07 AM: Compiling problem (target solver=OSQP).
(CVXP

4.9640211694779584e-11

In [12]:
coeff_approx_tight = coeff_var.value
print(coeff_approx_tight)
print(np.linalg.norm(coeff_approx_tight, ord=1))

[ 0.10925065 -1.          1.89074935]
3.0


As you can see, we have now obtained a comparatively higher L1-norm (which is still constrained by our choice of `max_l1_norm=3`). While the coefficient of the smallest $k_j$ value is still fairly small in magnitude, at least the other two provide a reasonable amount of mixing.

## Comparing our results

We will now briefly repeat the computations from the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial in order to assess the quality of the various coefficients.

In [13]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import generate_time_evolution_circuit

time = 8.0
circuits = []
for k in trotter_steps:
    circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(order=2, reps=k),
        time=time,
    )
    circuits.append(circ)

In [14]:
from qiskit.quantum_info import SparsePauliOp

L = 10
observable = SparsePauliOp.from_sparse_list([("Z", [i], 1 / L / 2) for i in range(L)], num_qubits=L)
print(observable)

SparsePauliOp(['IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIZII', 'IIIIIIZIII', 'IIIIIZIIII', 'IIIIZIIIII', 'IIIZIIIIII', 'IIZIIIIIII', 'IZIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.05+0.j, 0.05+0.j, 0.05+0.j, 0.05+0.j, 0.05+0.j, 0.05+0.j, 0.05+0.j,
 0.05+0.j, 0.05+0.j, 0.05+0.j])


In [15]:
from qiskit.primitives import StatevectorEstimator

estimator = StatevectorEstimator()
job = estimator.run([(circ, observable) for circ in circuits])
result = job.result()

In [16]:
evs = [res.data.evs for res in result]
print(evs)

[array(0.23799162), array(0.35754312), array(0.38649906)]


In [17]:
from scipy.linalg import expm

exp_H = expm(-1j * time * hamiltonian.to_matrix())

initial_state = np.zeros(exp_H.shape[0])
initial_state[0] = 1.0

time_evolved_state = exp_H @ initial_state

exact_obs = time_evolved_state.conj() @ observable.to_matrix() @ time_evolved_state

In [18]:
print("Analytical coeff solution:", evs @ coeff_analytical)
print("Plain approx.    solution:", evs @ coeff_approx_plain)
print("CLARABEL approx. solution:", evs @ coeff_approx_clarabel)
print("Tight approx.    solution:", evs @ coeff_approx_tight)
print("Exact            solution:", exact_obs.real)

Analytical coeff solution: 0.39548478559794153
Plain approx.    solution: 0.42928990901998165
CLARABEL approx. solution: 0.41833743748333746
Tight approx.    solution: 0.3992304700751579
Exact            solution: 0.40060242487900255


We can clearly see now, that the various approximate solutions can yield quite different results.
Luckily, testing out different expansion coefficients for a fixed choice of $k_j$ values does not require us to rerun our quantum experiments.
Therefore, we encourage you to play around and tweak your expansion coefficients when using the approximate solutions.